In [ ]:
import numpy as np
import pandas as pd
import pathlib as pa
import seaborn as sb
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
from datetime import datetime
from IPython.display import Markdown as md
from IPython.display import Math as m


In [ ]:
%matplotlib inline

In [ ]:
def extractParentDir(dataFileName):
    purePath = pathlib.PurePath(dataFileName)
    parentDir2 = purePath.parent.parent
    return parentDir2

In [ ]:
todayVal = datetime.today()
timeStampStr = todayVal.strftime("%y-%m-%d_%H-%M-%S.%f")
data_dir = r"C:\work\dev\dECMT_src\data_all\COVID19_Data\Current"
parentDir2 = extractParentDir(data_dir)
outputPath = Path(parentDir2) / "Output_Covid19_Analysis"/f"{timeStampStr}_bipartite"

# Renal Disease


In [400]:
comorbidFN = pa.Path(data_dir) / "REACT_Comorbidities_unpivoted.csv"
comorbidDF:pd.DataFrame = pd.read_csv(comorbidFN)

In [401]:
comorbidDF['COMORBIDITY'].unique()
# array(['COPD', 'ASTHMA', 'ILD', 'HYPERTENSION', 'THROMBOEMBOLIC',
#        'CARDIAC_D', 'RENAL_D', 'LIVER_D', 'NEURAL_D', 'META_SOLID_TUMOUR',
#        'MAGLINANT_NEO', 'DIABETES', 'OBESITY', 'HIV', 'RHEUMATO_D',
#        'DIMENTIA', 'IMMUNO'], dtype=object)


array(['COPD', 'ASTHMA', 'ILD', 'HYPERTENSION', 'THROMBOEMBOLIC',
       'CARDIAC_D', 'RENAL_D', 'LIVER_D', 'NEURAL_D', 'META_SOLID_TUMOUR',
       'MAGLINANT_NEO', 'DIABETES', 'OBESITY', 'HIV', 'RHEUMATO_D',
       'DIMENTIA', 'IMMUNO'], dtype=object)

In [402]:
comorbidDF['RENAL_D_PRESENT'] = (comorbidDF['COMORBIDITY'] == 'RENAL_D')
comorbidDF.head()

,STUDY_ID,COMORBIDITY,STATUS,RENAL_D_PRESENT
0,UHSCOVID_00989a42,COPD,1,False
1,UHSCOVID_0520de1c,COPD,1,False
2,UHSCOVID_0744a86c,COPD,1,False
3,UHSCOVID_0874d194,COPD,1,False
4,UHSCOVID_0b401459,COPD,1,False


In [403]:
comorbidDF['RENAL_D_PRESENT'].unique()


array([False,  True])

In [404]:
comorbidGrpDF = comorbidDF.groupby('STUDY_ID').agg(RENAL_D_PRESENT=('RENAL_D_PRESENT','any'))
comorbidGrpDF.head()



,RENAL_D_PRESENT
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,False
UHSCOVID_01add35c,False
UHSCOVID_01f7838d,True


# Died

In [ ]:

demogFN = pa.Path(data_dir) / "REACT_Demographics.csv"


In [ ]:
demogDF:pd.DataFrame = pd.read_csv(demogFN)

# **************************
lab_results_df = pd.read_csv(pa.Path(data_dir) / 'REACT_LabResults.csv')
# pharmacy_data_df = pd.read_csv(p.Path(data_dir) / 'REACT_PharmacyData.csv')
covid_test_df = pd.read_csv(pa.Path(data_dir) / 'REACT_UHSCOVIDTest_processed.csv')


In [ ]:
demogDF.describe()

In [ ]:
demogDF.columns

In [ ]:
demogDF.head()

In [ ]:
demogDF['Death'] = demogDF['DATE_OF_DEATH'].notna()

In [ ]:
demogDF2=demogDF.groupby('STUDY_ID').agg(Death=('Death','any'))
demogDF2

# Respiratory Support flag


In [ ]:
vitalSignsCategDF =  pd.read_csv(pa.Path(data_dir) / "REACT_Vitalsigns_Categorical.csv")


In [ ]:
vitalSignsCategDF[vitalSignsCategDF['PARAMETER'] == 'Respiratory Support']['VALUE'].unique()

In [ ]:
vitalSignsCategDF.head()

In [ ]:
vitalSignsCategDF.columns

In [ ]:
vitalSignsCategDF['PARAMETER'].unique()

In [ ]:
vitalSignsCategDF[vitalSignsCategDF['PARAMETER'] == 'Respiratory Support']['VALUE'].unique()

In [ ]:
vitalSignsCategDF['VALUE'].unique()


In [ ]:
invAirCategoryDict=dict()
invAirCategoryDict[0]=set(['Air - Not Supported'])
invAirCategoryDict[1]=set(
    ['Nasal Specs', 'Face Mask', 'Supine',
       'Left', 'Venturi Mask', 'Non-Rebreath Mask',
       'Invasive Ventilation', 'Paralysed', 'Right', 'Unable to Assess',
       'Prone', 'Deep Sedation', 'Unrousable', 'Moderate Sedation',
       'Cardiac Chair Position', 'Optiflow / Hi Flow', 'Drowsy',
       'Sat in chair', 'Positive', 'Alert and Calm',
       'NIV - CPAP face mask', 'Restless', 'Asleep',
       'NIV - CPAP full face mask', 'Negative', 'Light Sedation',
       'NIV - BIPAP face mask',
       'Declined Repositioning - Risks explained to patient', 'Agitated',
       'Combative', 'Very Agitated', 'NIV - CPAP nasal mask',
       'NIV - BIPAP nasal mask', 'NIV - BIPAP full face mask',
       'CPAP - Green', 'Trachy Mask', 'Off ward for procedure',
       'Pressure Support - Blue', 'Trachy Mask - Red',
       'Speaking Valve - Yellow'
     ]
        )

airCategoryDict = {v:k for k,lnames in invAirCategoryDict.items() for v in lnames}

In [ ]:
vitalSignsCategDF = vitalSignsCategDF[vitalSignsCategDF.PARAMETER=='Respiratory Support'][['STUDY_ID','UNITFROM_DATETIME','VALUE']]

In [ ]:
vitalSignsCategDF["AIR_SUPPORT"] = vitalSignsCategDF["VALUE"].map(airCategoryDict)

In [ ]:
vitalSignsCategDF["AIR_SUPPORT"] = vitalSignsCategDF["AIR_SUPPORT"] ==1

In [ ]:
vitalSignsCategDF[['AIR_SUPPORT','VALUE']].head()

In [ ]:
vitalSignsCategDF.dtypes

In [ ]:
vitalSignsCategDF.head()

In [ ]:
vitalSignsCategDF.columns



In [ ]:
# ['STUDY_ID', 'UNITFROM_DATETIME', 'VALUE', 'AIR_SUPPORT']
vitalSignsCategDF = vitalSignsCategDF.filter(items = ['STUDY_ID',  'AIR_SUPPORT'])

In [ ]:
vitalSignsCategDF['AIR_SUPPORT'].value_counts()

In [ ]:
vitalSignsCategDF.groupby('STUDY_ID').agg(AIR_SUPPORT=('AIR_SUPPORT','any'))


In [ ]:
# Association between Renal disease and Death

In [ ]:
demogDF2.head()

In [405]:
dfRes:pd.DataFrame = pd.merge(demogDF2, vitalSignsCategDF, on='STUDY_ID', how='inner')

In [406]:
dfRes = dfRes.reset_index()


In [407]:
dfRes.columns


Index(['index', 'STUDY_ID', 'Death', 'AIR_SUPPORT', 'RENAL_D_PRESENT'], dtype='object')

In [408]:
dfRes.head()

,index,STUDY_ID,Death,AIR_SUPPORT,RENAL_D_PRESENT
0,0,UHSCOVID_00989a42,False,False,False
1,1,UHSCOVID_00989a42,False,False,False
2,2,UHSCOVID_00989a42,False,False,False
3,3,UHSCOVID_00989a42,False,True,False
4,4,UHSCOVID_00989a42,False,False,False


In [381]:
md(f"Counts of stratified cases")

Counts of stratified cases

In [382]:
# crosstabRenalDeath = pd.crosstab(dfRes3['Death'],dfRes3['RENAL_D_PRESENT'],margins=True,margins_name='Total')
crosstabRenalDeath = pd.crosstab(dfRes['Death'],dfRes['AIR_SUPPORT'])
crosstabRenalDeath


AIR_SUPPORT,False,True
Death,,
False,8894,38300
True,2572,4906


In [385]:
crosstabRenalDeath2 = pd.crosstab(dfRes['Death'],dfRes['AIR_SUPPORT'],margins=True)
crosstabRenalDeath2



AIR_SUPPORT,False,True,All
Death,,,
False,8894,38300,47194
True,2572,4906,7478
All,11466,43206,54672


In [386]:
crosstabRenalDeathNorm = pd.crosstab(dfRes['Death'],dfRes['AIR_SUPPORT'],normalize=True,margins=True)
crosstabRenalDeathNorm

AIR_SUPPORT,False,True,All
Death,,,
False,0.162679,0.700541,0.863221
True,0.047044,0.089735,0.136779
All,0.209723,0.790277,1.000000


In [387]:
crosstabRenalDeath.index

Index([False, True], dtype='object', name='Death')

In [388]:
crosstabRenalDeath.columns

Index([False, True], dtype='object', name='AIR_SUPPORT')

In [389]:
crosstabRenalDeath.reset_index()
crosstabRenalDeath

AIR_SUPPORT,False,True
Death,,
False,8894,38300
True,2572,4906


In [390]:
# crosstabRenalDeath.loc['RENAL_D_PRESENT']

## Chi2 test on significance of renal disease for death outcome.


In [391]:
from scipy.stats import chi2_contingency

# defining the table
# data = [crosstabRenalDeath.loc['False',:].values().reshape(1,-1), [234, 242, 232]]
stat, p, dof, expected = chi2_contingency(crosstabRenalDeath)
print(f"stat = {stat}")
print(f"p = {p}")
print(f"dof = {dof}")
print(f"expected = {expected}")
md(f"Since p={p:.1g}<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.")

stat = 940.6571986212023
p = 1.424454350482319e-206
dof = 1
expected = [[ 9897.6881036 37296.3118964]
 [ 1568.3118964  5909.6881036]]


Since p=1e-206<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.

## Power of evidence

In [392]:
crosstabRenalDeathNorm

AIR_SUPPORT,False,True,All
Death,,,
False,0.162679,0.700541,0.863221
True,0.047044,0.089735,0.136779
All,0.209723,0.790277,1.000000


In [393]:
md(f" Conditional probabilities")

 Conditional probabilities

In [394]:
crosstabRenalDeath


AIR_SUPPORT,False,True
Death,,
False,8894,38300
True,2572,4906


In [ ]:
idx=pd.IndexSlice
R_T = crosstabRenalDeath.loc[:,idx[True]].sum()

In [ ]:
crosstabRenalDeathNorm

In [ ]:
R_T
P_R_T = crosstabRenalDeathNorm.loc['All',True]
P_D_T = crosstabRenalDeathNorm.loc[True,'All']
P_R_F = crosstabRenalDeathNorm.loc['All',False]
P_D_F = crosstabRenalDeathNorm.loc[False,'All']
PDTGRT = crosstabRenalDeathNorm.loc[True,True]/P_R_T
PDFGRT = 1-PDTGRT
# ********************************
PDFGRF = crosstabRenalDeathNorm.loc[False,False]/P_R_F
PDTGRF = 1-PDFGRF
# **********
PRTGDT =  crosstabRenalDeathNorm.loc[True,True]/P_D_T
PRFGDT = 1-PRTGDT
PRFGDF = crosstabRenalDeathNorm.loc[False,False]/P_D_F
PRTGDF = 1-PRFGDF
WoERTGD = PRTGDT/PRTGDF
WoERFGD = PRFGDT/PRFGDF
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRT={PDTGRT} PDFGRT={PDFGRT}")
print(f"WoERFGD={WoERFGD}")
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRF={PDTGRF} PDFGRT={PDFGRF}")
print(f"WoERTGD={WoERTGD}")

# Value of information (WoE)
The more is WoE different from 0 the more informative is renal disease for prediction of death, while positive value means support for selected result and negative value means the support of other result by evidence.



$$WoE(Evidence|Target) =100*log_{10}\Bigg  (\frac{P(Evidence|Target)}{P(Evidence|\neg Target)}\Bigg)$$

$$WoE(\neg Evidence|Target) =100*log_{10}\Bigg(\frac{P(\neg Evidence|Target)}{P(\neg Evidence|\neg Target)}\Bigg)$$

$$WoE \in (-\infty,\infty )$$$


In [395]:
md(f"Weight of Evidence, WoE(Renal_disease=Absent|Death) = {100*np.log10(WoERFGD)}")

Weight of Evidence, WoE(Renal_disease=Absent|Death) = 26.127516652027776

In [396]:
md(f"Weight of Evidence, WoE(Renal_disease=Present|Death) = {100*np.log10(WoERTGD)}")

Weight of Evidence, WoE(Renal_disease=Present|Death) = -9.236990271724672

In [397]:
RENAL_D_PRESENT = False
crosstabRenalDeath.loc[:,idx[RENAL_D_PRESENT]]

Death
False    8894
True     2572
dtype: int64

In [398]:
Died=False
crosstabRenalDeath.loc[Died,idx[RENAL_D_PRESENT]]


8894

In [399]:
crosstabRenalDeath.loc[False,idx[False]]


8894